In [1]:
pip install pymongo

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [2]:
# Biblioteca de VADER
!pip install vaderSentiment

Defaulting to user installation because normal site-packages is not writeable
     ---------------------------------------- 0.0/126.0 kB ? eta -:--:--
     -------------------------------------  122.9/126.0 kB 2.4 MB/s eta 0:00:01
     -------------------------------------- 126.0/126.0 kB 2.5 MB/s eta 0:00:00


In [3]:
import re
import pymongo
import nltk
from textblob import TextBlob
import matplotlib.pyplot as plt
#from wordcloud import WordCloud, STOPWORDS, ImageColorGenerator
#import numpy as np
import time
import string
from pprint import pprint
import pandas as pd
#import googletrans

In [4]:
!python --version

Python 3.11.4


In [5]:
from pymongo.mongo_client import MongoClient
from pymongo.server_api import ServerApi

In [6]:
uri = "mongodb+srv://ponerlaruta"
# Create a new client and connect to the server
client = MongoClient(uri, server_api=ServerApi('1'))
# Send a ping to confirm a successful connection
try:
    client.admin.command('ping')
    print("Pinged your deployment. You successfully connected to MongoDB!")
except Exception as e:
    print(e)

Pinged your deployment. You successfully connected to MongoDB!


In [7]:
# Conexión de la conexión  MongoDB
client = pymongo.MongoClient('mongodb+srv://ponerlaruta')

#Listar las bases de datos

databases = client.list_database_names()
for database_name in databases:
  print(f"Base de datos: {database_name}")


Base de datos: Cantantes_Colombianos
Base de datos: Deportistas_Colombianos
Base de datos: Empresas_Colombianas
Base de datos: Famosos
Base de datos: Futbol_Colombiano
Base de datos: Instituciones_del_Estado_Colombiano
Base de datos: Medios_Informaticos
Base de datos: Periodistas
Base de datos: Politicos_Colombianos
Base de datos: Universidades_Colombianas
Base de datos: admin
Base de datos: local


In [8]:
# Define el nombre de la colección a buscar
coleccion_a_buscar = "tweets_Actividad_R"

In [9]:
# Crear una lista
data = []

In [ ]:
# Itera sobre las bases de datos
for database_name in databases:
    # Conecta a la base de datos actual
    database = client[database_name]

    # Verifica si la colección existe en la base de datos actual
    if coleccion_a_buscar in database.list_collection_names():
        # Si existe, obtén todos los documentos de la colección
        collection = database[coleccion_a_buscar]
        tweets = collection.find()

        # Itera sobre los tweets en la colección
        for tweet in tweets:
            # Procesa cada tweet
            tweet_text = tweet["text"]
            data.append({'nombre_bd': database_name, 'text': tweet_text})
    else:
        print(f"La colección '{coleccion_a_buscar}' no existe en la base de datos '{database_name}'")


In [ ]:
df = pd.DataFrame(data)
df.head()

In [ ]:
# Definir la función para limpiar el texto
def limpiar_texto(text):
    text = text.lower()
    text = re.sub(r"@[\w\-]+", "", text) # Menciones
    text = re.sub(r"#", "", text) # Hashtag
    text = re.sub(r"^((https?://(www\.)?)|\www\.)\w{3,}\.[a-z]{2,3}$", "", text) # URL
    text = re.sub(r"https?://\S+|www\.\S+|t\.co/[A-Za-z0-9]+", "", text) # URL cortas de Twitter
    #text = re.sub(r"[^\x00-\x7F]+", "", text) # Emijis
    text = re.sub(r"[\U0001F600-\U0001F64F\U0001F300-\U0001F5FF\U0001F700-\U0001F77F\U0001F780-\U0001F7FF\U0001F800-\U0001F8FF\U0001F900-\U0001F9FF\U0001FA00-\U0001FA6F\U0001FA70-\U0001FAFF\U0001FB00-\U0001FBFF\U0001FC00-\U0001FCFF\U0001FD00-\U0001FDFF\U0001FE00-\U0001FEFF\U0001FF00-\U0001FFFF\U00002600-\U000026FF\U00002700-\U000027BF\U0001F004-\U0001F0CF\U0001F170-\U0001F251\U0001F004-\U0001F0CF\U0001F300-\U0001F5FF\U0001F600-\U0001F64F\U0001F650-\U0001F67F\U0001F680-\U0001F6FF\U0001F700-\U0001F77F\U0001F780-\U0001F7FF\U0001F800-\U0001F8FF\U0001F900-\U0001F9FF\U0001FA00-\U0001FA6F\U0001FA70-\U0001FAFF\U0001FB00-\U0001FBFF\U0001FC00-\U0001FCFF\U0001FD00-\U0001FDFF\U0001FE00-\U0001FEFF\U0001FF00-\U0001FFFF\U00002600-\U000026FF\U00002700-\U000027BF\U0001F004-\U0001F0CF\U0001F170-\U0001F251]", "", text)
    text = re.sub(r"\$\d+(?:,\d{3})*(?:\.\d+?)", "", text) # Valores de Dinero, tiene en cuenta el símbolo $ y la separación del dígito ya sea por "," o "."
    text = re.sub(r"((\d)+/+(\d+[a-zA-Z]))", "", text) # Número/NúmeroLetra 173/2022c
    text = re.sub(r"\b\d+(?:\.\d+)?\b", "", text) # Números enteros y decimales
    text = re.sub(r'^rt[\s]:+', '', text)

    #text = re.sub(r"^rt @([\w\-]+):", "", text) # Retweets de Usuarios
    return text

In [ ]:
df = pd.DataFrame(data)
df.head(20)

In [ ]:
df['clean_text'] = df['text'].apply(limpiar_texto)

In [ ]:
df['clean_text']

In [ ]:
df.head(20)

In [ ]:
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer

# Léxico en Español
analyzer = SentimentIntensityAnalyzer()

In [ ]:
df.head()

In [ ]:
# Función para Calcular el sentimiento
def get_polarity(text):
  sentiment = analyzer.polarity_scores(text)

#Determina la polaridad del sentimiento
  if sentiment['compound'] > 0:
    return 'Positivo'
  elif sentiment['compound'] < 0:
    return 'Negativo'
  else:
    return 'Neutral'


In [ ]:
df['sentimiento'] = df['clean_text'].apply(get_polarity)

In [ ]:
df

In [ ]:
df.head(10)

In [ ]:
# Tabla que muestra la frecuencia de cada sentimiento para cada valor único de nombre_bd
tabla = pd.crosstab(df['nombre_bd'], df['sentimiento'])
print(tabla)

In [ ]:
# Gráfico de barras apiladas para visualizar la distribución
tabla.plot(kind='bar', stacked = True, figsize=(10, 6))
plt.title('Distribución de Sentimientos por Base de Datos')
plt.xlabel('Nombre de la Base de Datos')
plt.ylabel('Frecuencia')
plt.legend(title='Sentimiento', loc='upper right')
plt.show()

In [ ]:
# Gráfico de barras agrupadas para visualizar la distribución
tabla.plot(kind='bar', figsize=(10, 6))
plt.title('Distribución de Sentimientos por Base de Datos')
plt.xlabel('Nombre de la Base de Datos')
plt.ylabel('Frecuencia')
plt.legend(title='Sentimiento', loc='upper right')
plt.show()

In [ ]:
# Agrupo los Tweets por el nombre de la Base de Datos
grupos = df.groupby('nombre_bd')

# Almaceno en un diccionario los DataFrames separados por el nombre de la base de datos
tweets_separados = {}

# Creo un Dataframe por cada base de datos

for nombre_db, grupo in grupos:
  tweets_separados[nombre_db] = grupo.copy()



In [ ]:
cantantes = tweets_separados['Cantantes_Colombianos']
deportistas = tweets_separados['Deportistas_Colombianos']
empresas = tweets_separados['Empresas_Colombianas']
famosos = tweets_separados['Famosos']
futbol = tweets_separados['Futbol_Colombiano']
instituciones = tweets_separados['Instituciones_del_Estado_Colombiano']
medios = tweets_separados['Medios_Informaticos']
periodistas = tweets_separados['Periodistas']
politicos = tweets_separados['Politicos_Colombianos']
universidades = tweets_separados['Universidades_Colombianas']

In [ ]:
# Contar el número de tweets positivos
tweets_positivos = df[df['sentimiento'] == 'Positivo']

# Calcular el porcentaje de tweets positivos
porcentaje_positivos = (len(tweets_positivos) / len(df)) * 100

print(f"Porcentaje de tweets positivos: {porcentaje_positivos:.2f}%")

In [ ]:
# Contar el número de tweets negativos
tweets_negativos = df[df['sentimiento'] == 'Negativo']

# Calcular el porcentaje de tweets negativos
porcentaje_negativos = (len(tweets_negativos) / len(df)) * 100

print(f"Porcentaje de tweets negativos: {porcentaje_negativos:.2f}%")

In [ ]:
#Exportar el DataFrame a Excel
df.to_excel('resultados_tweets_VADER.xlsx', index= False)